# 📊 TRAF-GNN Data Exploration

This notebook explores the METR-LA traffic dataset:
- Load and inspect raw data
- Analyze data quality and missing values
- Visualize temporal traffic patterns
- Explore spatial relationships

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Imports successful!")

## 1. Load Dataset

In [ ]:
# Data paths
data_dir = Path('../data/raw')
h5_file = data_dir / 'metr-la.h5'
adj_file = data_dir / 'adj_mx.pkl'
sensor_ids_file = data_dir / 'graph_sensor_ids.txt'
sensor_locs_file = data_dir / 'graph_sensor_locations.csv'

# Check if files exist
print("Checking data files...")
print(f"H5 file: {h5_file.exists()}")
print(f"Adjacency matrix: {adj_file.exists()}")
print(f"Sensor IDs: {sensor_ids_file.exists()}")
print(f"Sensor locations: {sensor_locs_file.exists()}")

In [ ]:
# Load traffic speed data
with h5py.File(h5_file, 'r') as f:
    # Print available keys
    print("Available keys in H5 file:")
    print(list(f.keys()))
    
    # Load data (usually stored under 'speed' or 'data' key)
    # Adjust key name if different
    data = f['speed'][:] if 'speed' in f.keys() else f[list(f.keys())[0]][:]

print(f"\n📊 Data shape: {data.shape}")
print(f"   Timesteps: {data.shape[0]:,}")
print(f"   Sensors: {data.shape[1]}")
print(f"   Duration: ~{data.shape[0] * 5 / 60 / 24:.1f} days")

In [ ]:
# Load adjacency matrix
with open(adj_file, 'rb') as f:
    sensor_ids, sensor_id_to_ind, adj_mx = pickle.load(f, encoding='latin1')

print(f"\n🗺️ Adjacency matrix shape: {adj_mx.shape}")
print(f"   Number of sensors: {len(sensor_ids)}")
print(f"   Number of edges: {np.sum(adj_mx > 0)}")
print(f"   Graph density: {np.sum(adj_mx > 0) / (adj_mx.shape[0] * adj_mx.shape[1]):.4f}")

In [ ]:
# Load sensor locations (if available)
try:
    sensor_locs = pd.read_csv(sensor_locs_file)
    print("\n📍 Sensor locations:")
    print(sensor_locs.head())
except:
    print("\n⚠️  Sensor locations file not found or has different format")
    sensor_locs = None

## 2. Data Quality Analysis

In [ ]:
# Check for missing values
missing_mask = np.isnan(data)
missing_pct = np.mean(missing_mask) * 100

print(f"\n📉 Missing Values Analysis:")
print(f"   Total missing: {missing_pct:.2f}%")
print(f"   Missing entries: {np.sum(missing_mask):,}")

# Missing values per sensor
missing_per_sensor = np.mean(missing_mask, axis=0) * 100
print(f"\n   Sensors with >10% missing: {np.sum(missing_per_sensor > 10)}")
print(f"   Max missing in a sensor: {np.max(missing_per_sensor):.2f}%")

In [ ]:
# Visualize missing data pattern
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Missing values per sensor
axes[0].bar(range(len(missing_per_sensor)), sorted(missing_per_sensor, reverse=True))
axes[0].set_xlabel('Sensor (sorted by missingness)')
axes[0].set_ylabel('Missing %')
axes[0].set_title('Missing Values per Sensor')
axes[0].axhline(y=10, color='r', linestyle='--', label='10% threshold')
axes[0].legend()

# Missing values over time
missing_per_time = np.mean(missing_mask, axis=1) * 100
axes[1].plot(missing_per_time, alpha=0.7)
axes[1].set_xlabel('Timestep')
axes[1].set_ylabel('Missing %')
axes[1].set_title('Missing Values Over Time')

plt.tight_layout()
plt.show()

In [ ]:
# Traffic speed statistics
valid_data = data[~missing_mask]

print(f"\n🚗 Traffic Speed Statistics (mph):")
print(f"   Mean: {np.mean(valid_data):.2f}")
print(f"   Median: {np.median(valid_data):.2f}")
print(f"   Std Dev: {np.std(valid_data):.2f}")
print(f"   Min: {np.min(valid_data):.2f}")
print(f"   Max: {np.max(valid_data):.2f}")

# Speed distribution
plt.figure(figsize=(10, 5))
plt.hist(valid_data, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Speed (mph)')
plt.ylabel('Frequency')
plt.title('Distribution of Traffic Speeds')
plt.axvline(np.mean(valid_data), color='r', linestyle='--', label=f'Mean: {np.mean(valid_data):.1f}')
plt.legend()
plt.show()

## 3. Temporal Pattern Analysis

In [ ]:
# Select a few sensors for visualization
sample_sensors = [0, 50, 100, 150, 200]  # Sample 5 sensors

# Plot traffic speed over time for sample sensors
plt.figure(figsize=(15, 6))
for sensor in sample_sensors:
    plt.plot(data[:1000, sensor], alpha=0.7, label=f'Sensor {sensor}')

plt.xlabel('Timestep (5-min intervals)')
plt.ylabel('Speed (mph)')
plt.title('Traffic Speed Time Series (First 1000 timesteps)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Daily pattern (assuming 5-min intervals: 288 intervals per day)
intervals_per_day = 288

# Average speed by time of day across all sensors and days
n_days = data.shape[0] // intervals_per_day
daily_data = data[:n_days * intervals_per_day].reshape(n_days, intervals_per_day, -1)
avg_daily_pattern = np.nanmean(daily_data, axis=(0, 2))

# Time labels (hours)
hours = np.arange(24)
hourly_speed = [np.nanmean(avg_daily_pattern[i*12:(i+1)*12]) for i in range(24)]

plt.figure(figsize=(12, 5))
plt.plot(hours, hourly_speed, marker='o', linewidth=2)
plt.xlabel('Hour of Day')
plt.ylabel('Average Speed (mph)')
plt.title('Average Traffic Speed by Hour of Day')
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24, 2))
plt.axvspan(7, 9, alpha=0.2, color='red', label='Morning Rush')
plt.axvspan(17, 19, alpha=0.2, color='orange', label='Evening Rush')
plt.legend()
plt.show()

print("\n⏰ Peak Traffic Insights:")
print(f"   Slowest hour: {hours[np.argmin(hourly_speed)]}:00 ({min(hourly_speed):.1f} mph)")
print(f"   Fastest hour: {hours[np.argmax(hourly_speed)]}:00 ({max(hourly_speed):.1f} mph)")

## 4. Spatial Analysis

In [ ]:
# Visualize adjacency matrix
plt.figure(figsize=(10, 8))
plt.imshow(adj_mx, cmap='viridis', aspect='auto')
plt.colorbar(label='Connection Weight')
plt.xlabel('Sensor ID')
plt.ylabel('Sensor ID')
plt.title('Physical Road Network Adjacency Matrix')
plt.show()

# Degree distribution
degrees = np.sum(adj_mx > 0, axis=1)
plt.figure(figsize=(10, 5))
plt.hist(degrees, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Node Degree')
plt.ylabel('Frequency')
plt.title('Degree Distribution of Road Network')
plt.show()

print(f"\n📊 Network Statistics:")
print(f"   Average degree: {np.mean(degrees):.2f}")
print(f"   Max degree: {np.max(degrees)}")
print(f"   Min degree: {np.min(degrees)}")

In [ ]:
# Sensor map (if location data available)
if sensor_locs is not None and 'latitude' in sensor_locs.columns:
    plt.figure(figsize=(12, 8))
    plt.scatter(sensor_locs['longitude'], sensor_locs['latitude'], 
                c=np.nanmean(data, axis=0), cmap='RdYlGn', s=50, alpha=0.6)
    plt.colorbar(label='Average Speed (mph)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Sensor Locations Colored by Average Speed')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("⚠️  Sensor location data not available for mapping")

## 5. Correlation Analysis

In [ ]:
# Calculate correlation between sensors (sample subset for speed)
sample_size = 50  # Sample first 50 sensors
subset_data = data[:, :sample_size]

# Fill NaN values with column mean for correlation calculation
subset_filled = np.copy(subset_data)
for i in range(subset_filled.shape[1]):
    col_mean = np.nanmean(subset_filled[:, i])
    subset_filled[np.isnan(subset_filled[:, i]), i] = col_mean

corr_matrix = np.corrcoef(subset_filled.T)

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, 
            vmin=-1, vmax=1, square=True)
plt.title(f'Correlation Matrix (First {sample_size} Sensors)')
plt.xlabel('Sensor ID')
plt.ylabel('Sensor ID')
plt.show()

print(f"\n🔗 Correlation Insights:")
print(f"   Average correlation: {np.mean(corr_matrix[np.triu_indices_from(corr_matrix, k=1)]):.3f}")
print(f"   Max correlation: {np.max(corr_matrix[np.triu_indices_from(corr_matrix, k=1)]):.3f}")

## 6. Summary & Next Steps

In [ ]:
print("="*60)
print("📝 DATA EXPLORATION SUMMARY")
print("="*60)
print(f"\n✅ Dataset: METR-LA")
print(f"   - Sensors: {data.shape[1]}")
print(f"   - Timesteps: {data.shape[0]:,} (5-min intervals)")
print(f"   - Duration: ~{data.shape[0] * 5 / 60 / 24:.1f} days")
print(f"   - Missing data: {missing_pct:.2f}%")
print(f"\n✅ Traffic Characteristics:")
print(f"   - Average speed: {np.mean(valid_data):.1f} mph")
print(f"   - Speed range: {np.min(valid_data):.1f} - {np.max(valid_data):.1f} mph")
print(f"\n✅ Network Structure:")
print(f"   - Nodes: {adj_mx.shape[0]}")
print(f"   - Edges: {np.sum(adj_mx > 0)}")
print(f"   - Avg degree: {np.mean(degrees):.1f}")
print(f"\n📋 Next Steps:")
print("   1. Handle missing values (interpolation/imputation)")
print("   2. Normalize traffic speeds")
print("   3. Create train/validation/test splits")
print("   4. Build multi-view graphs")
print("="*60)